# Masking (Screening)を使って、ユニバースを作成する

このNotebookは、
* [TUTORIAL2 Pipeline](https://www.quantopian.com/tutorials/pipeline) の [Masking](https://www.quantopian.com/tutorials/pipeline#lesson7)
* [TUTORIAL6 Writing a Contest Algorithm](https://www.quantopian.com/tutorials/contest) の [Liquid Universe](https://www.quantopian.com/tutorials/contest#lesson3)

を参考に、編集・今回の勉強会に沿った形で展開しています。


## はじめに
デフォルトの状態（空の状態）のパイプラインを実行すると、

* 「Quantopianで利用可能な全銘柄」

がデフォルトで選択されてきました。

このとき、約8,000銘柄の情報が含まれています。しかし、この中には投資対象としてふさわしくない銘柄（1日にほとんど取引が成立しないような銘柄など。日本でいえば、名古屋証券取引所に単独上場しているような銘柄などをイメージすると分かりやすいでしょう。）も多く含まれていることでしょう。そのような銘柄の場合、仮にアルゴリズムが「買い」シグナルを発生させたとしても買うことができなかったり、仮に買うことができても約定値段が高くなってしまい、期待したようなパフォーマンスが上げられない可能性が高くなります。

そのような銘柄を除外するために、一定の条件でフィルタリングして銘柄を絞ることが一般的です。 一定の基準を持って投資対象銘柄を絞り込む行為のことを金融の世界では「スクリーニング」とか「マスキング」と呼びます。個人的な印象としては、前者の方が一般的に利用されている気がします。

そして、スクリーニングを通過した銘柄群のことを「ユニバース」と呼びます。ユニバースから漏れた銘柄への投資は行いません。本Notebokでは、まず、Quantopianが用意している組み込みのユニバースをパイプラインに適用します。

次に、スクリーニング条件を自作し、Quantopianの全データに対して独自のスクリーニングを行う方法を説明します。

In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing

# 組み込みユニバース
from quantopian.pipeline.experimental import QTradableStocksUS
# ユニバースを自作するために利用する組み込みファクターの例
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume



In [ ]:
# 02-03.パイプラインの要素を作成する、と同様のファクターを作ると仮定
def make_pipeline():
    
    
    # 米国株データベースから終値を取得して、10日間移動平均を計算する
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    # 終値
    latest_close = USEquityPricing.close.latest

    return Pipeline(
        columns={
            '10_day_mean_close' :  mean_close_10,
            'latest_close_price': latest_close
        },
        # QTradableStocksUS でスクリーニング
        screen=QTradableStocksUS()
    )


# パイプライン実行
result = run_pipeline(make_pipeline(), '2018-11-01', '2018-11-01')
result


## 組み込みユニバースQTradableStocksUS

Quantopianでは、独自の選定基準（QTradableStocksUSについての詳細は[ここ](https://www.quantopian.com/posts/working-on-our-best-universe-yet-qtradablestocksus)）に基づきQTradableStocksUSというユニバースを提供しています。

例えば、、、
* 時価総額が５億ドル以上
* 過去200日の出来高のメディアン（中央値）が250万ドル以上
* 過去200日間の約定データが存在すること
などが基準となっています。

パイプラインに対し、QTradableStocksUSでスクリーニングをするためには、以下のようにコードを追加します。

出力結果は約8700レコードから、2191レコードにまで減少しました。
流動性の高い銘柄のみが選択されていることもあり、終値がN/Aとなっている銘柄が見当たらなくなっています。

## 自分でスクリーニング条件を作成する。

先ほどの例では、Quantopianが定めたスクリーニング条件によって予め作成された"QTradableStocksUS"を用いてパイプラインをマスキングしユニバースを作成ましたが、勿論自分でスクリーニング条件を定めて銘柄を絞り込むことも可能です。

例えば以下のように、
* high_dollar_volume : 過去30日間の平均売買代金が市場全体の上位10%に含まれ、かつ、
* top_open_price : そのなかで直近の始値が高い50銘柄に入り、かつ、
* hig_close_price = そのなかで直近の終値が上位10%

と、いうような複雑な条件でスクリーニング条件を作成することも可能です。
このようなマスキングの仕方の場合、ユニバースに含まれる構成銘柄は頻繁に入れ替わるかもしれません。一般的に、ユニバースはある程度静的なもの（定期的な見直しとか）ですので、こういう使い方の場合は、ユニバースというよりも純粋な「条件検索」に近いかもしれません。

上記スクリーニングで出力される銘柄数は、５（= 50 x 0.1）だと思われます。早速確認してみましょう。

In [ ]:
def make_pipeline():

    # Dollar volume factor（組み込みファクターを利用）
    dollar_volume = AverageDollarVolume(window_length=30)

    # High dollar volume filter
    high_dollar_volume = dollar_volume.percentile_between(90,100)

    # Top open securities filter (high dollar volume securities)
    top_open_price = USEquityPricing.open.latest.top(50, mask=high_dollar_volume)

    # Top percentile close price filter (high dollar volume, top 50 open price)
    high_close_price = USEquityPricing.close.latest.percentile_between(90, 100, mask=top_open_price)

    return Pipeline(
        screen=high_close_price
    )

result = run_pipeline(make_pipeline(), '2018-11-01', '2018-11-01')
result


In [ ]:
reset_df = result.reset_index()

for equity in reset_df["level_1"]:
    print equity.asset_name


# コンテストに参加するには、QTradableStocksUSが必須!

QtradableStocksUSに含まれない銘柄でアルゴリズムを組むと、その時々の偶然性によってパフォーマンスが左右される可能性が上がります。
「いつ、いかなるときでも安定した結果が出ること」がアルゴリズムとして望ましい性質であるのは明らかでしょう。QTradableStoskcUSに含まれる銘柄というのは、流動性が高く、安定して売買できる銘柄に限定します。Quantopianのコンテストに参加する場合、アルゴリズムがQTradableStocksUSをユニバースにすることを義務化しているのには、このような背景があるものと思います。

このことは、01-01.基礎知識で説明した「ファクターとアノマリーの違い」に通じるものがあります。
今回の勉強会においては、QTradableStaocksUSをユニバースに用いて分析していきます。